In [1]:
import pandas as pd
import csv
import os, os.path
from sklearn.preprocessing import LabelEncoder
import sklearn as sk  
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm  
from sklearn.linear_model import LogisticRegression 
from sklearn.cluster import KMeans
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.cluster import SpectralClustering

# import numpy as np
# from sklearn import preprocessing
# import matplotlib.pyplot as plt 
# import seaborn as sns
# from numpy import array


In [2]:
#creating my dataframe
mydata=pd.DataFrame({"Db": [], "Hz": []})
labels=pd.DataFrame({"Male": [], "Feeling": [], "Person":[]})

In [3]:
students=['\\mario','\\giuliano','\\cristina'] #directory of records of each student
for i in students:
    
    cwd = os.getcwd() #current working directory
    sd = cwd + str(i) #student directory
    lab = pd.read_csv(str(sd)+'\\labels.csv', sep=';') #read the labels file
    labels=pd.concat([labels,lab], ignore_index=True)
    
    for k in range(1,len(os.listdir(sd))): #runs the cycle with respect to number of records for each student
        
        read = pd.read_csv(str(sd)+'\\sl Registrazione '+str(k)+'.csv', sep = ',') #read data  
        
        Db=read["DecibelSource"].dropna()  #clean from NAs
        Hz=read["PitchSensor"].dropna()    
        
        Db=list(Db)#make them to list to easier work later
        Hz=list(Hz)
        
        mydata=mydata.append({'Db' : Db , 'Hz' : Hz} , ignore_index=True)

    
result=pd.concat([mydata,labels], sort=False, axis=1) #merge labels and data

In [4]:
result.head()

,Db,Hz,Male,Feeling,Person
0,"[51.509708077318315, 49.206463639207875, 53.09...","[83.5404052734375, 83.5404052734375, 83.540405...",1.0,angry,other
1,"[55.10859961906438, 51.864688959738814, 51.652...","[291.0975477430556, 331.6312255859375, 331.631...",0.0,happy,other
2,"[29.52738094783193, 27.660100580824576, 25.288...","[268.5400390625, 230.9423828125, 230.942382812...",1.0,steady,other
3,"[33.119446531464924, 28.733594908973828, 33.64...","[419.1162109375, 419.1162109375, 419.116210937...",1.0,sad,other
4,"[19.45318383545405, 18.038502361992837, 16.372...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 339.233398...",0.0,angry,other


In [5]:
#see if there are problem with classes
print(set(result.Person))
print(set(result.Feeling))

{'other ', 'mario', 'other', 'giuliano', 'cristina'}
{'happy', 'steady', 'serious', 'sad', 'angry', 'happy '}


In [6]:
#fix them
for i in range(len(result)):
    if result['Feeling'][i]=='happy ':
        result['Feeling'][i]='happy'
    if result['Feeling'][i]=='serious':
        result['Feeling'][i]='steady'
    if result['Person'][i]=='other ':
        result['Person'][i]='other'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Categorical encoding

In [7]:
##transform every cathegorical var into numbers
le=LabelEncoder()

cat=['Feeling','Person']

for i in cat:
    result[i]=le.fit_transform(result[i].astype(str))

In [8]:
result.head()

,Db,Hz,Male,Feeling,Person
0,"[51.509708077318315, 49.206463639207875, 53.09...","[83.5404052734375, 83.5404052734375, 83.540405...",1.0,0,3
1,"[55.10859961906438, 51.864688959738814, 51.652...","[291.0975477430556, 331.6312255859375, 331.631...",0.0,1,3
2,"[29.52738094783193, 27.660100580824576, 25.288...","[268.5400390625, 230.9423828125, 230.942382812...",1.0,3,3
3,"[33.119446531464924, 28.733594908973828, 33.64...","[419.1162109375, 419.1162109375, 419.116210937...",1.0,2,3
4,"[19.45318383545405, 18.038502361992837, 16.372...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 339.233398...",0.0,0,3


# Cleaning

In [9]:
#are the feeling balanced?
print(str(len(result.loc[result["Feeling"]==0]))+' angry records')
print(str(len(result.loc[result["Feeling"]==1]))+' happy records')
print(str(len(result.loc[result["Feeling"]==2]))+' sad records')
print(str(len(result.loc[result["Feeling"]==3]))+' steady records')
print('')
print(str(len(result.loc[result["Male"]==0]))+' female records')
print(str(len(result.loc[result["Male"]==1]))+' male records')

16 angry records
33 happy records
17 sad records
64 steady records

65 female records
65 male records


In [10]:
#evaluate mininum len
mini=100000000000000000 
for i in range(len(result)):
    now=len(result["Hz"][i]) #see lenght of first HZ vector
#     print(str(i)+ " " +str(now)) #print number of vector and his lenght
    if now<mini: #see if is the shortest vector
        mini=now #if shortest, update minimum value
# mini

In [11]:
#same len for all

for i in range(len(result)):
    result["Db"][i]=result["Db"][i][0:mini]

for i in range(len(result)):
    result["Hz"][i]=result["Hz"][i][0:mini]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [12]:
#selecting only male and female to study their properties
male=result.loc[result["Male"]==1]
male=male.reset_index(drop=True)  #setting new indexes for the new DF
female=result.loc[result["Male"]==0]
female=female.reset_index(drop=True)

In [13]:
##let's see how the means differs
somma=0
for i in range(len(male)):
    somma+=mean(male["Db"][i])
print("mean for DB in male is "+str(round(somma/(len(male)+1),2)))

somma=0
for i in range(len(male)):
    somma+=mean(male["Hz"][i])
print("mean for HZ in male is " + str(round(somma/(len(male)+1),2)))

##now for female..
somma=0
for i in range(len(female)):
    somma+=mean(female["Db"][i])
print("mean for DB in female is "+str(round(somma/(len(female)+1),2)))

somma=0
for i in range(len(female)):
    somma+=mean(female["Hz"][i])
print("mean for HZ in female is " +str(round(somma/(len(female)+1),2)))

mean for DB in male is 46.3
mean for HZ in male is 305.66
mean for DB in female is 41.42
mean for HZ in female is 282.85


In [14]:
#Creating our X matrix
cols=['Db','Hz']#selecting only hz and db data
mat=[] #empty matrix
for c in cols:
    for k in range(mini):
        l=[] #empty array
        for i in range(len(result)):
            l.append(result[c][i][k]) #filling the array making it the column we are going to use
        mat.append(l) #append our column to the matrix
X=pd.DataFrame(mat) #make it a pandas df
X=X.T #transpose it
X=pd.concat([X,result.Person],axis=1)#add person column

In [16]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,Person
0,51.509708,49.206464,53.098951,52.469692,53.167057,52.176484,48.654212,53.256975,52.688264,56.267311,...,214.801432,214.801432,128.659220,128.659220,128.659220,128.659220,78.327759,78.327759,78.327759,3
1,55.108600,51.864689,51.652674,47.638640,49.127333,48.002193,53.917669,54.711934,54.078948,51.856219,...,328.022705,328.022705,77.165527,77.165527,77.165527,332.609030,332.609030,332.609030,332.609030,3
2,29.527381,27.660101,25.288264,20.859257,22.952716,26.111523,39.505439,35.678947,34.919236,32.231507,...,222.426758,222.426758,222.426758,122.093913,122.093913,122.093913,64.099609,64.099609,64.099609,3
3,33.119447,28.733595,33.640379,45.195911,46.322613,38.580593,42.709851,39.776593,30.642523,24.077781,...,128.453762,184.490601,184.490601,184.490601,184.490601,168.398966,168.398966,168.398966,168.398966,3
4,19.453184,18.038502,16.372480,15.836151,23.020714,23.356387,21.816119,22.132527,21.352715,19.915726,...,527.344727,527.344727,527.344727,1027.164103,1027.164103,1027.164103,1027.164103,497.380534,497.380534,3


In [57]:
#define Y. SHUTDOWN THE Y YOU DON'T WANT TO USE
y=result.Feeling 
# y=result.Male

#splitting train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Random forest
## on Gender

In [15]:
"""#define y
y_G=result.Male #y for gender
y_F=result.Feeling #y for feeling"""

In [47]:
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)  
RF.fit(X_train, y_train)  
RF.predict(X_test)

array([0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 1., 1.])

In [49]:
for i in range(len(y_test)):
    print(y_test.reset_index(drop=True)[i],end= "-")
    print(RF.predict(X_test)[i], end=' ')
    if RF.predict(X_test)[i]==y_test.reset_index(drop=True)[i]:
        print(' well done! \n')
    else:
        print('\n')

0.0-0.0  well done! 

0.0-0.0  well done! 

0.0-0.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

0.0-0.0  well done! 

0.0-0.0  well done! 

1.0-1.0  well done! 

0.0-0.0  well done! 

0.0-0.0  well done! 

0.0-0.0  well done! 

1.0-0.0 

0.0-0.0  well done! 

0.0-0.0  well done! 

0.0-0.0  well done! 

0.0-0.0  well done! 

1.0-0.0 

0.0-0.0  well done! 

0.0-1.0 

1.0-1.0  well done! 

0.0-0.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

0.0-0.0  well done! 

0.0-0.0  well done! 

1.0-0.0 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

0.0-0.0  well done! 

0.0-0.0  well done! 

1.0-1.0  well done! 

0.0-1.0 

1.0-0.0 

0.0-0.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

0.0-1.0 



# Random forest
## on Feeling

In [58]:
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)  
RF.fit(X_train, y_train)  
RF.predict(X_test)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [60]:
for i in range(len(y_test)):
    print(y_test.reset_index(drop=True)[i],end= "-")
    print(RF.predict(X_test)[i], end='')
    if RF.predict(X_test)[i]==y_test.reset_index(drop=True)[i]:
        print(' well done! \n')
    else:
        print('\n')

3-3 well done! 

3-3 well done! 

3-3 well done! 

1-3

3-3 well done! 

1-3

3-3 well done! 

1-3

3-3 well done! 

3-3 well done! 

2-3

1-3

0-3

0-3

3-3 well done! 

0-3

2-3

0-3

1-3

2-3

1-3

1-3

3-3 well done! 

3-3 well done! 

3-3 well done! 

3-3 well done! 

3-3 well done! 

3-3 well done! 

1-3

0-3

1-3

1-3

3-3 well done! 

3-3 well done! 

1-3

3-3 well done! 

3-3 well done! 

1-3

3-3 well done! 

3-3 well done! 

3-3 well done! 

2-3

2-3



# Support vector machine 
## on Gender

In [50]:
SVM = svm.LinearSVC()  
SVM.fit(X_train, y_train)  
SVM.predict(X_test) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


array([1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
       1., 1., 0., 0., 1., 1., 1., 1., 1.])

In [52]:
for i in range(len(y_test)):
    print(y_test.reset_index(drop=True)[i],end= "-")
    print(SVM.predict(X_test)[i], end=' ')
    if SVM.predict(X_test)[i]==y_test.reset_index(drop=True)[i]:
        print(' well done! \n')
    else:
        print('\n')

0.0-1.0 

0.0-1.0 

0.0-1.0 

1.0-0.0 

1.0-0.0 

1.0-1.0  well done! 

0.0-1.0 

0.0-0.0  well done! 

1.0-1.0  well done! 

0.0-1.0 

0.0-1.0 

0.0-0.0  well done! 

1.0-1.0  well done! 

0.0-1.0 

0.0-1.0 

0.0-1.0 

0.0-1.0 

1.0-0.0 

0.0-1.0 

0.0-1.0 

1.0-0.0 

0.0-1.0 

1.0-0.0 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-0.0 

0.0-0.0  well done! 

0.0-1.0 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-0.0 

1.0-1.0  well done! 

0.0-0.0  well done! 

0.0-1.0 

1.0-1.0  well done! 

0.0-0.0  well done! 

1.0-0.0 

0.0-1.0 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

0.0-1.0 



# Support vector machine 
## on Feeling

In [61]:
SVM = svm.SVC(decision_function_shape="ovo")  
SVM.fit(X_train, y_train)  
SVM.predict(X_test) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [62]:
for i in range(len(y_test)):
    print(y_test.reset_index(drop=True)[i],end= "-")
    print(SVM.predict(X_test)[i], end=' ')
    if SVM.predict(X_test)[i]==y_test.reset_index(drop=True)[i]:
        print(' well done! \n')
    else:
        print('\n')

3-3  well done! 

3-3  well done! 

3-3  well done! 

1-3 

3-3  well done! 

1-3 

3-3  well done! 

1-3 

3-3  well done! 

3-3  well done! 

2-3 

1-3 

0-3 

0-3 

3-3  well done! 

0-3 

2-3 

0-3 

1-3 

2-3 

1-3 

1-3 

3-3  well done! 

3-3  well done! 

3-3  well done! 

3-3  well done! 

3-3  well done! 

3-3  well done! 

1-3 

0-3 

1-3 

1-3 

3-3  well done! 

3-3  well done! 

1-3 

3-3  well done! 

3-3  well done! 

1-3 

3-3  well done! 

3-3  well done! 

3-3  well done! 

2-3 

2-3 



# Logistic regression
## on Gender

In [53]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)  
LR.predict(X_test)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


array([1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
       1., 1., 0., 0., 1., 1., 1., 1., 1.])

In [54]:
for i in range(len(y_test)):
    print(y_test.reset_index(drop=True)[i],end= "-")
    print(LR.predict(X_test)[i], end=' ')
    if LR.predict(X_test)[i]==y_test.reset_index(drop=True)[i]:
        print(' well done! \n')
    else:
        print('\n')

0.0-1.0 

0.0-1.0 

0.0-1.0 

1.0-0.0 

1.0-0.0 

1.0-1.0  well done! 

0.0-1.0 

0.0-0.0  well done! 

1.0-1.0  well done! 

0.0-1.0 

0.0-1.0 

0.0-0.0  well done! 

1.0-1.0  well done! 

0.0-1.0 

0.0-1.0 

0.0-0.0  well done! 

0.0-1.0 

1.0-0.0 

0.0-1.0 

0.0-1.0 

1.0-0.0 

0.0-1.0 

1.0-0.0 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-0.0 

0.0-0.0  well done! 

0.0-1.0 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-0.0 

1.0-1.0  well done! 

0.0-0.0  well done! 

0.0-1.0 

1.0-1.0  well done! 

0.0-0.0  well done! 

1.0-0.0 

0.0-1.0 

1.0-1.0  well done! 

1.0-1.0  well done! 

1.0-1.0  well done! 

0.0-1.0 



# Logistic regression
## on Feeling

In [63]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)  
LR.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


array([3, 0, 3, 3, 3, 2, 1, 1, 1, 0, 1, 3, 1, 3, 2, 3, 0, 2, 3, 3, 2, 3,
       3, 1, 1, 3, 0, 1, 3, 1, 0, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2])

In [64]:
for i in range(len(y_test)):
    print(y_test.reset_index(drop=True)[i],end= "-")
    print(LR.predict(X_test)[i], end='')
    if LR.predict(X_test)[i]==y_test.reset_index(drop=True)[i]:
        print(' well done! \n')
    else:
        print('\n')

3-3 well done! 

3-0

3-3 well done! 

1-3

3-3 well done! 

1-2

3-1

1-1 well done! 

3-1

3-0

2-1

1-3

0-1

0-3

3-2

0-3

2-0

0-2

1-3

2-3

1-2

1-3

3-3 well done! 

3-1

3-1

3-3 well done! 

3-0

3-1

1-3

0-1

1-0

1-3

3-3 well done! 

3-3 well done! 

1-3

3-2

3-2

1-3

3-3 well done! 

3-3 well done! 

3-3 well done! 

2-3

2-2 well done! 



# Clustering

In [22]:
##set the kmean model
kmeans = KMeans(init='k-means++', n_init=10, n_clusters=2, random_state=0).fit(list(result["Db"]))

In [23]:
##predict the gender from Db for each line in the matrix
count=0 #to count how many times it's right
for i in range(len(result)):
    pred=kmeans.predict(list([result["Hz"][i]])) #say in which cluster is the datapoint db, i
    true=result["Male"][i] #the true Male value of line i 
    
    print(str(pred)+" "+str(true))
    if pred==true: #if he get the real value
        count+=1
print(str(count)+"/"+str(len(result)))     #print how many times it predict  

[0] 1.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 1.0
[0] 0.0
[0] 1.0
[0] 0.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 1.0
[0] 0.0
[0] 1.0


## Boolean encoding

In [24]:
"""#let's now encode the cathegorical variables
toAdd=pd.get_dummies(result['Person']).reset_index(drop=True)
result=pd.concat([result,toAdd], axis=1, sort=False)

toAdd=pd.get_dummies(result['Feeling']).reset_index(drop=True)
result=pd.concat([result,toAdd], axis=1, sort=False)

result=result.drop(['Person','Feeling'],axis=1)"""

"#let's now encode the cathegorical variables\ntoAdd=pd.get_dummies(result['Person']).reset_index(drop=True)\nresult=pd.concat([result,toAdd], axis=1, sort=False)\n\ntoAdd=pd.get_dummies(result['Feeling']).reset_index(drop=True)\nresult=pd.concat([result,toAdd], axis=1, sort=False)\n\nresult=result.drop(['Person','Feeling'],axis=1)"

In [25]:
"""result.head()"""

'result.head()'

In [26]:
"""#Create a new matrix 26x123x2 in which we have
#26 lines--->the number of records
#123 cols--->the pitch&Db recorded by SJapp
#2 elements for each cell---> Db, Hz
newVec=[]#gonna be our final result
for i in range(len(result)):
    temp=[]#gonna have 26 of this
    for k in range(len(result["Db"][i])):#fill our temp with a 123x2 matrix
        d=result["Db"][i][k]
        h=result["Hz"][i][k]
        temp.append([d,h])#for each cycle it writes a cell with 2 values (db,hz)
    newVec.append(temp)
newVec"""

'#Create a new matrix 26x123x2 in which we have\n#26 lines--->the number of records\n#123 cols--->the pitch&Db recorded by SJapp\n#2 elements for each cell---> Db, Hz\nnewVec=[]#gonna be our final result\nfor i in range(len(result)):\n    temp=[]#gonna have 26 of this\n    for k in range(len(result["Db"][i])):#fill our temp with a 123x2 matrix\n        d=result["Db"][i][k]\n        h=result["Hz"][i][k]\n        temp.append([d,h])#for each cycle it writes a cell with 2 values (db,hz)\n    newVec.append(temp)\nnewVec'

In [27]:
"""#trying to do with Hz and Db toghether
kmeans = KMeans(n_clusters=2, random_state=0).fit(newVec)"""

'#trying to do with Hz and Db toghether\nkmeans = KMeans(n_clusters=2, random_state=0).fit(newVec)'

In [28]:
"""count=0
for i in range(len(result)):
    pred=kmeans.predict(list([result["Hz"][i]+result["Db"][i]]))
    true=result["Male"][i]
    
    print(str(pred)+" "+str(true))
    if pred==true:
        count+=1
print(str(count)+"/"+str(len(result)))"""      

'count=0\nfor i in range(len(result)):\n    pred=kmeans.predict(list([result["Hz"][i]+result["Db"][i]]))\n    true=result["Male"][i]\n    \n    print(str(pred)+" "+str(true))\n    if pred==true:\n        count+=1\nprint(str(count)+"/"+str(len(result)))'

In [29]:
"""###change the values 0 in pitch with the mean 
#NOT NECESSARY! DON'T RUN IT
for i in range(len(result)):
    for k in range(len(result["Hz"][i])):
        if result["Hz"][i][k]==0:
            result["Hz"][i][k]=mean(result["Hz"][i])"""

'###change the values 0 in pitch with the mean \n#NOT NECESSARY! DON\'T RUN IT\nfor i in range(len(result)):\n    for k in range(len(result["Hz"][i])):\n        if result["Hz"][i][k]==0:\n            result["Hz"][i][k]=mean(result["Hz"][i])'